## Loading data into GCS

In [19]:
from google.cloud import storage
from google.oauth2 import service_account
import os
from datetime import datetime, timedelta

In [6]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']

'/Users/ns/github-repos/energy-dashboard/energy-dashboard.json'

In [7]:
credentials = service_account.Credentials.from_service_account_file(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])
project_id = 'ml-energy-dashboard'

In [9]:
bucket_name = 'ml-energy-dashboard-raw-data'
storage_client = storage.Client(credentials=credentials, project=project_id)

if storage_client.bucket(bucket_name):
    print(f'Bucket exists: {bucket_name}')
else:
    print(f'Creating bucket: {bucket_name}')
    storage_client.create_bucket(bucket_name)


Bucket exists: ml-energy-dashboard-raw-data


In [12]:
bucket=storage_client.get_bucket(bucket_name)



In [11]:
from entsoe import EntsoePandasClient
import pandas as pd

In [42]:
print(time_pairs[-1])
load_entsoe(time_pairs[-1])

('20200308T2300', '20200309T2300')


2020-03-09 00:00:00+01:00    24556.0
2020-03-09 01:00:00+01:00    23070.0
2020-03-09 02:00:00+01:00    21989.0
2020-03-09 03:00:00+01:00    21614.0
2020-03-09 04:00:00+01:00    21757.0
2020-03-09 05:00:00+01:00    22605.0
2020-03-09 06:00:00+01:00    25764.0
2020-03-09 07:00:00+01:00    29636.0
2020-03-09 08:00:00+01:00    32126.0
2020-03-09 09:00:00+01:00    33268.0
2020-03-09 10:00:00+01:00    33580.0
2020-03-09 11:00:00+01:00    33448.0
2020-03-09 12:00:00+01:00    33283.0
2020-03-09 13:00:00+01:00    32947.0
2020-03-09 14:00:00+01:00    31841.0
2020-03-09 15:00:00+01:00    31141.0
2020-03-09 16:00:00+01:00    30437.0
2020-03-09 17:00:00+01:00    30197.0
2020-03-09 18:00:00+01:00    30034.0
2020-03-09 19:00:00+01:00    32487.0
2020-03-09 20:00:00+01:00    33947.0
2020-03-09 21:00:00+01:00    33071.0
2020-03-09 22:00:00+01:00    30494.0
2020-03-09 23:00:00+01:00    27820.0
dtype: float64

In [36]:
client = EntsoePandasClient(api_key = os.environ['ENTSOE_TOKEN'])
def load_entsoe(time_pair):
    data = client.query_load("ES",
                  start=pd.Timestamp(time_pair[0], tz='UTC'), 
                  end=pd.Timestamp(time_pair[1], tz='UTC'))
    return data

In [17]:
data

2020-03-08 00:00:00+01:00    24985.0
2020-03-08 01:00:00+01:00    23210.0
2020-03-08 02:00:00+01:00    21964.0
2020-03-08 03:00:00+01:00    21315.0
2020-03-08 04:00:00+01:00    21078.0
2020-03-08 05:00:00+01:00    21215.0
2020-03-08 06:00:00+01:00    21555.0
2020-03-08 07:00:00+01:00    21738.0
2020-03-08 08:00:00+01:00    22702.0
2020-03-08 09:00:00+01:00    24647.0
2020-03-08 10:00:00+01:00    26384.0
2020-03-08 11:00:00+01:00    26802.0
2020-03-08 12:00:00+01:00    26621.0
2020-03-08 13:00:00+01:00    26791.0
2020-03-08 14:00:00+01:00    26445.0
2020-03-08 15:00:00+01:00    25292.0
2020-03-08 16:00:00+01:00    24512.0
2020-03-08 17:00:00+01:00    24375.0
2020-03-08 18:00:00+01:00    24930.0
2020-03-08 19:00:00+01:00    27636.0
2020-03-08 20:00:00+01:00    29788.0
2020-03-08 21:00:00+01:00    29801.0
2020-03-08 22:00:00+01:00    28619.0
2020-03-08 23:00:00+01:00    26465.0
dtype: float64

In [34]:
#generate a list of dates for the last 90 days
end = datetime.today()
start = datetime.today() + timedelta(-90)
delta = end-start

date_list = [(start + timedelta(i)).strftime('%Y%m%d') for i in range(delta.days+1)]

time_pairs = list()

for i in range(delta.days+1):
    begin_time = (start + timedelta(i-1)).strftime('%Y%m%d')
    begin_time = f'{begin_time}T2300'
    end_time = (start + timedelta(i)).strftime('%Y%m%d')
    end_time = f'{end_time}T2300'
    
    time_pairs.append((begin_time, end_time))
    

print(start)
print(end)
print(time_pairs)

2019-12-11 21:10:46.190587
2020-03-10 21:10:46.190520
[('20191210T2300', '20191211T2300'), ('20191211T2300', '20191212T2300'), ('20191212T2300', '20191213T2300'), ('20191213T2300', '20191214T2300'), ('20191214T2300', '20191215T2300'), ('20191215T2300', '20191216T2300'), ('20191216T2300', '20191217T2300'), ('20191217T2300', '20191218T2300'), ('20191218T2300', '20191219T2300'), ('20191219T2300', '20191220T2300'), ('20191220T2300', '20191221T2300'), ('20191221T2300', '20191222T2300'), ('20191222T2300', '20191223T2300'), ('20191223T2300', '20191224T2300'), ('20191224T2300', '20191225T2300'), ('20191225T2300', '20191226T2300'), ('20191226T2300', '20191227T2300'), ('20191227T2300', '20191228T2300'), ('20191228T2300', '20191229T2300'), ('20191229T2300', '20191230T2300'), ('20191230T2300', '20191231T2300'), ('20191231T2300', '20200101T2300'), ('20200101T2300', '20200102T2300'), ('20200102T2300', '20200103T2300'), ('20200103T2300', '20200104T2300'), ('20200104T2300', '20200105T2300'), ('2020010

In [39]:
#download all dates in the time pairs and upload to GCS
folder='raw-days'
bucket=storage_client.get_bucket(bucket_name)

for time_pair in time_pairs:
    s = time_pair[0]
    e = time_pair[1]
    file_name = f'es-energy-demand-{s}-{e}'
    blob = bucket.blob(f'{folder}/{file_name}')
    
    data = load_entsoe(time_pair)
    
    blob.upload_from_string(data.to_json())
    print(f'Uploaded {file_name}...')

Uploaded es-energy-demand-20191210T2300-20191211T2300...
Uploaded es-energy-demand-20191211T2300-20191212T2300...
Uploaded es-energy-demand-20191212T2300-20191213T2300...
Uploaded es-energy-demand-20191213T2300-20191214T2300...
Uploaded es-energy-demand-20191214T2300-20191215T2300...
Uploaded es-energy-demand-20191215T2300-20191216T2300...
Uploaded es-energy-demand-20191216T2300-20191217T2300...
Uploaded es-energy-demand-20191217T2300-20191218T2300...
Uploaded es-energy-demand-20191218T2300-20191219T2300...
Uploaded es-energy-demand-20191219T2300-20191220T2300...
Uploaded es-energy-demand-20191220T2300-20191221T2300...
Uploaded es-energy-demand-20191221T2300-20191222T2300...
Uploaded es-energy-demand-20191222T2300-20191223T2300...
Uploaded es-energy-demand-20191223T2300-20191224T2300...
Uploaded es-energy-demand-20191224T2300-20191225T2300...
Uploaded es-energy-demand-20191225T2300-20191226T2300...
Uploaded es-energy-demand-20191226T2300-20191227T2300...
Uploaded es-energy-demand-20191

### Adding a date interpolation function

In [8]:
#copy of cloud function to download and store data
def store_day_load(request):
    from entsoe import EntsoePandasClient
    from datetime import datetime, timedelta
    import pandas as pd
    import numpy as np
    from google.cloud import storage
    
    FOLDER = 'raw-days'
    BUCKET = 'ml-energy-dashboard-raw-data'
    
    def generate_dates():

        start = (datetime.today()+timedelta(-2)).strftime('%Y%m%d')
        start = f'{start}T2300'

        end = (datetime.today()+timedelta(-1)).strftime('%Y%m%d')
        end =f'{end}T2300'

        date = (datetime.today()+timedelta(-1)).strftime('%Y%m%d')
        
        return start, end, date
    
    def gcs_file_name(start, end):
        return f'es-energy-demand-raw-{start}-{end}'
    
    def query_entsoe_load(start, end):
        
        client = EntsoePandasClient(api_key='909addb7-e4ae-4702-acc7-6b4f4fd9667b')
        data = client.query_load("ES", 
            start=pd.Timestamp(start, tz='UTC'), 
            end=pd.Timestamp(end, tz='UTC'))
        
        return data
    
    def check_fill_nans(data, date):
        print(date)
        idx = pd.date_range(start=f'{date}T0000', end=f'{date}T2300', freq='H').tz_localize('Europe/Madrid')
        data = data.append(pd.Series(np.NaN, index = idx.difference(data.index))).sort_index().interpolate()
        
        return data
        
    def upload_data_to_gcs(data, bucket_name, folder_name, file_name):
        
        storage_client = storage.Client()
        bucket = storage_client.get_bucket(bucket_name)
        blob = bucket.blob(f'{folder_name}/{file_name}')
        blob.upload_from_string(data.to_json())
        
    
    params = request#.get_json()
    
    if "get_data" in params and params['get_data']:
        
        start_time, end_time, date = generate_dates()
        
        file_name = gcs_file_name(start_time, end_time)
        
        data = query_entsoe_load(start_time, end_time)
        
        data = check_fill_nans(data, date)
        
        upload_data_to_gcs(data, BUCKET, FOLDER, file_name)

In [9]:
request = {"get_data": 'true'}
data = store_day_load(request)


20200310


In [10]:
data['20200310']

2020-03-10 00:00:00+01:00    26025.0
2020-03-10 01:00:00+01:00    24464.0
2020-03-10 02:00:00+01:00    23602.0
2020-03-10 03:00:00+01:00    22989.0
2020-03-10 04:00:00+01:00    23066.0
2020-03-10 05:00:00+01:00    23649.0
2020-03-10 06:00:00+01:00    26358.0
2020-03-10 07:00:00+01:00    29863.0
2020-03-10 08:00:00+01:00    32001.0
2020-03-10 09:00:00+01:00    32997.0
2020-03-10 10:00:00+01:00    33063.0
2020-03-10 11:00:00+01:00    32864.5
2020-03-10 12:00:00+01:00    32666.0
2020-03-10 13:00:00+01:00    32583.0
2020-03-10 14:00:00+01:00    31553.0
2020-03-10 15:00:00+01:00    30862.0
2020-03-10 16:00:00+01:00    30254.0
2020-03-10 17:00:00+01:00    29920.0
2020-03-10 18:00:00+01:00    29832.0
2020-03-10 19:00:00+01:00    32071.0
2020-03-10 20:00:00+01:00    33985.0
2020-03-10 21:00:00+01:00    33242.0
2020-03-10 22:00:00+01:00    30704.0
2020-03-10 23:00:00+01:00    27988.0
dtype: float64

In [59]:
import numpy as np

In [64]:
idx = pd.date_range(start='20200310T0000', end='20200310T2300', periods=24).tz_localize('Europe/Madrid')
data.append(pd.Series(np.NaN, index = idx.difference(data.index))).sort_index().interpolate()

2020-03-10 00:00:00+01:00    26025.0
2020-03-10 01:00:00+01:00    24464.0
2020-03-10 02:00:00+01:00    23602.0
2020-03-10 03:00:00+01:00    22989.0
2020-03-10 04:00:00+01:00    23066.0
2020-03-10 05:00:00+01:00    23649.0
2020-03-10 06:00:00+01:00    26358.0
2020-03-10 07:00:00+01:00    29863.0
2020-03-10 08:00:00+01:00    32001.0
2020-03-10 09:00:00+01:00    32997.0
2020-03-10 10:00:00+01:00    33063.0
2020-03-10 11:00:00+01:00    32864.5
2020-03-10 12:00:00+01:00    32666.0
2020-03-10 13:00:00+01:00    32583.0
2020-03-10 14:00:00+01:00    31553.0
2020-03-10 15:00:00+01:00    30862.0
2020-03-10 16:00:00+01:00    30254.0
2020-03-10 17:00:00+01:00    29920.0
2020-03-10 18:00:00+01:00    29832.0
2020-03-10 19:00:00+01:00    32071.0
2020-03-10 20:00:00+01:00    33985.0
2020-03-10 21:00:00+01:00    33242.0
2020-03-10 22:00:00+01:00    30704.0
2020-03-10 23:00:00+01:00    27988.0
dtype: float64